In [1]:
%load_ext autoreload
%autoreload 2

# Experiments with TEASER-S model and simulated feedback

In [2]:
from pathlib import Path
import numpy as np

import teaser.util as util
import teaser.evaluation as evaluation

from teaser.algorithm.teaser_s import TEASER_S

## Datasets

In [3]:
## Uncomment corresponding directory.
# DATA_DIR = Path('../../data/ML20M/')
DATA_DIR = Path('../../data/Amazon Video Games/')
# DATA_DIR = Path('../../data/Publiq/')

In [4]:
# Shouldn't need to change this info
INTERACTIONS_TRAIN_PATH = DATA_DIR / 'interactions.train.csv'
INTERACTIONS_VALIN_PATH = DATA_DIR / 'interactions.valin.csv'
INTERACTIONS_VALOUT_PATH = DATA_DIR / 'interactions.valout.csv'
INTERACTIONS_TESTIN_PATH = DATA_DIR / 'interactions.testin.csv'
INTERACTIONS_TESTOUT_PATH = DATA_DIR / 'interactions.testout.csv'

METADATA_PATH = DATA_DIR / 'features.csv'

ITEM_ID = 'itemId'
USER_ID = 'userId'

## Define algorithm and hyperparameter ranges

In [5]:
ALG = TEASER_S

# ML20M
# L2_1 = 100
# L2_2 = 100000000
# RHO = 1000
# DELTA = 0
# MAX_ITER = 10

# AVG
L2_1 = 500
L2_2 = 2000
RHO = 10000
DELTA = 0
MAX_ITER = 15

# Publiq
# L2_1 = 500
# L2_2 = 50000
# RHO = 1000
# DELTA = 0
# MAX_ITER = 10

HYPERPARAMS = {'l2_1': L2_1, 'l2_2': L2_2, 'rho': RHO, 'delta': DELTA,'max_iterations': MAX_ITER}
HYPERPARAMS

{'l2_1': 500, 'l2_2': 2000, 'rho': 10000, 'delta': 0, 'max_iterations': 15}

## Parse data

In [6]:
S, tags = util.parse_metadata(METADATA_PATH, item_id=ITEM_ID)
n, t = S.shape  # amount of items and tags

X = util.parse_interactions(INTERACTIONS_TRAIN_PATH, item_id=ITEM_ID, user_id=USER_ID, shape_items=n)

Xtest_in = util.parse_interactions(INTERACTIONS_TESTIN_PATH, item_id=ITEM_ID, user_id=USER_ID, shape_items=n)
Xtest_out = util.parse_interactions(INTERACTIONS_TESTOUT_PATH, item_id=ITEM_ID, user_id=USER_ID, shape_items=n)

In [7]:
# # debug
# d_items = 1000
# S = S[:d_items,:]
# tags = tags[:d_items]
# X = X[:,:d_items]
# Xtest_in = Xtest_in[:,:d_items]
# Xtest_out = Xtest_out[:,:d_items]

## Evaluate model with optimal hyperparams on test set

In [8]:
# Precomput XTX and decomposition of XTX, since it's needed for all hyperparameters combinations and doesn't change
XTX = (X.T @ X).toarray()
p, U = np.linalg.eigh(XTX)
fit_params = {'XTX': XTX, 'p': p, 'U': U}

In [9]:
%%time
alg = ALG(**HYPERPARAMS)
alg.fit(X, S, **fit_params)

Decompose DTD


norm E 0.23864950670571944
diag norm: 0.8440507114634
diag_diff: 0.6289088997417903

norm E 0.5704712827218316
diag norm: 3.728183425136433
diag_diff: 2.02122323535349
rho * change E: 6296.200026916258

norm E 0.2667538081291606
diag norm: 1.4636078164986799
diag_diff: 0.18757223161752626
rho * change E: 3552.952125580751

norm E 0.19416957791849423
diag norm: 0.6230917325705085
diag_diff: 0.4746943621119989
rho * change E: 1225.122357944679

norm E 0.1787821796717859
diag norm: 0.2800925197691525
diag_diff: 0.4077096024242727
rho * change E: 478.1456812793898

norm E 0.175073385759121
diag norm: 0.135628695034427
diag_diff: 0.2796397360991062
rho * change E: 204.72038729668847

norm E 0.17383447808650704
diag norm: 0.08053744170116534
diag_diff: 0.17521917209038024
rho * change E: 101.80123880490429

norm E 0.1732798070941364
diag norm: 0.06524970832786571
diag_diff: 0.10515933618859427
rho * change E: 65.4786047402696

norm E 0.17298296221186246
diag norm: 0.06436037105232084
diag_di

In [11]:
print("Performance on test set with simulation")
for POS_FEEDBACK in [1, 2]:
    for STRENGTH in [3]:
        print("Pos feedback count:", POS_FEEDBACK, "with strength (out of 5):", STRENGTH)
        score = evaluation.simulate(alg, Xtest_in, Xtest_out, S, pos_feedback=POS_FEEDBACK, strength=STRENGTH, repeats=3)
    print()

Performance on test set with simulation
Pos feedback count: 1 with strength (out of 5): 3
Evaluating with 2000 users
Average Recall@20 0.281
Average Recall@100 0.522
Average nDCG@100 0.185

Pos feedback count: 2 with strength (out of 5): 3
Evaluating with 2000 users
Average Recall@20 0.478
Average Recall@100 0.714
Average nDCG@100 0.294

